## Homework

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2022-05-12 12:57:38--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 64.233.191.138, 64.233.191.101, 64.233.191.113, ...
Connecting to docs.google.com (docs.google.com)|64.233.191.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/s33iqgku9fh6sdnuddphkpv750j52qi1/1652360250000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2022-05-12 12:57:40--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/s33iqgku9fh6sdnuddphkpv750j52qi1/1652360250000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 209.85.146.132, 2607:f8b0:4001:c1f::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.goog

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [96]:
import pandas as pd

In [97]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [98]:
df.head(3)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000


In [99]:
df.shape

(20000, 292)

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [100]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [101]:
num_df = df[num_columns]

In [102]:
for i in range(len(df.columns)):
  missings = df[df.columns[i]].isna().sum()
  p_c = missings / len(df) * 100
  print('col:', df.columns[i], 'missings:', missings, 'percentage', p_c)

col: id missings: 0 percentage 0.0
col: timestamp missings: 0 percentage 0.0
col: full_sq missings: 0 percentage 0.0
col: life_sq missings: 4103 percentage 20.515
col: floor missings: 113 percentage 0.565
col: max_floor missings: 6303 percentage 31.514999999999997
col: material missings: 6303 percentage 31.514999999999997
col: build_year missings: 8905 percentage 44.525
col: num_room missings: 6303 percentage 31.514999999999997
col: kitch_sq missings: 6303 percentage 31.514999999999997
col: state missings: 8907 percentage 44.535000000000004
col: product_type missings: 0 percentage 0.0
col: sub_area missings: 0 percentage 0.0
col: area_m missings: 0 percentage 0.0
col: raion_popul missings: 0 percentage 0.0
col: green_zone_part missings: 0 percentage 0.0
col: indust_part missings: 0 percentage 0.0
col: children_preschool missings: 0 percentage 0.0
col: preschool_quota missings: 4279 percentage 21.395
col: preschool_education_centers_raion missings: 0 percentage 0.0
col: children_school 

Пропусков в данных много и просто удалить их нельзя. В sklearn есть штука, которая может нам помочь. Правда все что она делает это заполняет пропуски агрегированными значениями.

In [103]:
from sklearn.impute import SimpleImputer
import numpy as np

In [104]:
num_df.shape

(20000, 266)

In [105]:
num_df.shape[0]*(0.8)

16000.0

In [110]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(num_df)
num_df = imputer.transform(num_df)
num_df = pd.DataFrame(data=num_df, columns=num_columns)

In [115]:
X_train, y_train = num_df.drop('price', axis=1)[:16000], df['price'][:16000]
X_test, y_test = num_df.drop('price', axis=1)[16000:], df['price'][16000:]

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [116]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [117]:
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train.values, y_train.values)

DecisionTreeRegressor(random_state=42)

Проверьте качество на отложенной выборке.

In [118]:
preds = model.predict(X_test.values)
print(f'RMSE: {mean_squared_error(y_test, preds)**0.5}')

RMSE: 3856144.3247972373


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [119]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [120]:
month = df.timestamp.dt.month
day = df.timestamp.dt.weekday
fl_max_f = df.floor / df.max_floor
ksq_fsq = df.kitch_sq / df.full_sq

In [121]:
df.month = month
df.day = day
df.fl_max_f = fl_max_f
df.ksq_fsq = ksq_fsq

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [126]:
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [127]:
num_df = df[num_columns]

In [128]:
num_df.shape

(20000, 268)

In [129]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(num_df)
num_df = imputer.transform(num_df)
num_df = pd.DataFrame(data=num_df, columns=num_columns)

In [130]:
X_train, y_train = num_df.drop('price', axis=1)[:16000], df['price'][:16000]
X_test, y_test = num_df.drop('price', axis=1)[16000: ], df['price'][16000: ]

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [131]:
parametrs_dt = {'max_depth': [2, 3, 5, 10],
             'min_samples_split': [2, 5, 7], 
             'min_samples_leaf': [2, 5, 7]
             }
model = DecisionTreeRegressor(random_state=42)
gs_dt = GridSearchCV(model, parametrs_dt, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
gs_dt.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [2, 3, 5, 10],
                         'min_samples_leaf': [2, 5, 7],
                         'min_samples_split': [2, 5, 7]},
             scoring='neg_mean_squared_error', verbose=1)

In [132]:
preds_dt = gs_dt.predict(X_test)
print(f'RMSE: {mean_squared_error(y_test, preds_dt)**0.5}')

RMSE: 3123370.3260512007


In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
param_grid = {'n_estimators': [100, 200],
              'min_samples_split':[2, 5],
              'min_samples_leaf': [2, 5],
              'max_depth': [5, 10]}

model = RandomForestRegressor()
gs_rf = GridSearchCV(model,
                     param_grid,
                     cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

gs_rf.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [2, 5],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='neg_mean_squared_error', verbose=1)

In [ ]:
preds_rf = gs_rf.predict(X_test)
print(f'RMSE: {mean_squared_error(y_test, preds_rf)**0.5}')

RMSE: 2745664.40071762


In [ ]:
# !pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [66]:
model = CatBoostRegressor()

grid = {'learning_rate': [0.03, 0.1],
        'depth': [5, 10],
        'l2_leaf_reg': [3, 5, 7]}

gs_CatBoost = model.grid_search(grid, 
                                X=X_train, 
                                y=y_train, 
                                plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Выходные данные были обрезаны до нескольких последних строк (5000).
26:	learn: 4856926.9514066	test: 4789340.0522014	best: 4789340.0522014 (26)	total: 14.5s	remaining: 8m 43s
27:	learn: 4771633.3997479	test: 4704462.1576317	best: 4704462.1576317 (27)	total: 15.1s	remaining: 8m 43s
28:	learn: 4689053.5808954	test: 4621340.3341539	best: 4621340.3341539 (28)	total: 15.6s	remaining: 8m 42s
29:	learn: 4614289.8367186	test: 4545717.8637508	best: 4545717.8637508 (29)	total: 16.1s	remaining: 8m 42s
30:	learn: 4537119.6688475	test: 4469972.2108228	best: 4469972.2108228 (30)	total: 16.7s	remaining: 8m 41s
31:	learn: 4471884.7292684	test: 4408590.9817369	best: 4408590.9817369 (31)	total: 17.2s	remaining: 8m 40s
32:	learn: 4403049.6163201	test: 4342107.4819461	best: 4342107.4819461 (32)	total: 17.8s	remaining: 8m 40s
33:	learn: 4336576.6277609	test: 4276127.1734965	best: 4276127.1734965 (33)	total: 18.3s	remaining: 8m 39s
34:	learn: 4274627.2088235	test: 4215730.0672178	best: 4215730.0672178 (34)	

In [67]:
gs_CatBoost['params']

{'depth': 10, 'l2_leaf_reg': 3, 'learning_rate': 0.03}

Считалось бесконечно долго, но все же результат лучше, чем просто у моделей выше. К тому же теперь знаем оптимальные параметры.

In [68]:
# from google.colab import output
# output.disable_custom_widget_manager()

In [69]:
best_model = CatBoostRegressor(depth=10, l2_leaf_reg=3, learning_rate=0.03)
best_model.fit(X_train, y_train)

0:	learn: 4526020.8854912	total: 1.04s	remaining: 17m 22s
1:	learn: 4458159.4872013	total: 2.18s	remaining: 18m 9s
2:	learn: 4394956.3151917	total: 3.43s	remaining: 19m 1s
3:	learn: 4326485.5198875	total: 4.49s	remaining: 18m 38s
4:	learn: 4265755.9413031	total: 5.57s	remaining: 18m 28s
5:	learn: 4203952.2198791	total: 6.55s	remaining: 18m 5s
6:	learn: 4143549.5238434	total: 7.6s	remaining: 17m 57s
7:	learn: 4087174.9050962	total: 8.54s	remaining: 17m 39s
8:	learn: 4034572.4444234	total: 9.81s	remaining: 18m
9:	learn: 3982794.3654011	total: 10.9s	remaining: 18m 1s
10:	learn: 3934576.2615832	total: 11.7s	remaining: 17m 33s
11:	learn: 3887406.9827162	total: 12.3s	remaining: 16m 53s
12:	learn: 3841730.3651007	total: 12.9s	remaining: 16m 19s
13:	learn: 3793680.8418911	total: 13.5s	remaining: 15m 51s
14:	learn: 3747803.5655823	total: 14.1s	remaining: 15m 25s
15:	learn: 3707836.3677070	total: 14.7s	remaining: 15m 4s
16:	learn: 3664943.8359465	total: 15.3s	remaining: 14m 45s
17:	learn: 362652

In [70]:
preds = best_model.predict(X_test)
print(f'RMSE: {mean_squared_error(y_test, preds)**0.5}')

RMSE: 2633401.308510579


### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [71]:
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [72]:
invest_train_X = df.loc[np.where(df.product_type == 'Investment')[0], num_columns].drop('price', axis = 1)
invest_train_Y = df.loc[(np.where(df.product_type == 'Investment')[0]), 'price']

owner_train_X = df.loc[(np.where(df.product_type == 'OwnerOccupier')[0]), num_columns].drop('price', axis = 1)
owner_train_Y = df.loc[(np.where(df.product_type == 'OwnerOccupier')[0]), 'price']

In [74]:
model_invest = CatBoostRegressor(depth=10, l2_leaf_reg=3, learning_rate=0.03)
model_invest.fit(invest_train_X.values, invest_train_Y.values)

0:	learn: 4821714.9855170	total: 963ms	remaining: 16m 2s
1:	learn: 4753851.0442860	total: 1.99s	remaining: 16m 32s
2:	learn: 4689027.0479389	total: 2.9s	remaining: 16m 3s
3:	learn: 4629922.7496860	total: 3.76s	remaining: 15m 35s
4:	learn: 4570041.2816372	total: 4.53s	remaining: 15m 2s
5:	learn: 4511419.9373407	total: 5.39s	remaining: 14m 52s
6:	learn: 4454818.3598082	total: 6.24s	remaining: 14m 45s
7:	learn: 4398629.1968742	total: 7.18s	remaining: 14m 50s
8:	learn: 4343633.9413376	total: 7.98s	remaining: 14m 38s
9:	learn: 4288472.5575715	total: 9.06s	remaining: 14m 56s
10:	learn: 4236773.3023733	total: 9.96s	remaining: 14m 55s
11:	learn: 4188380.4987330	total: 10.5s	remaining: 14m 25s
12:	learn: 4142530.3464371	total: 11.1s	remaining: 13m 59s
13:	learn: 4095081.5750996	total: 11.6s	remaining: 13m 37s
14:	learn: 4049333.2701305	total: 12.1s	remaining: 13m 16s
15:	learn: 4008458.8354039	total: 12.7s	remaining: 13m
16:	learn: 3967686.3078607	total: 13.2s	remaining: 12m 45s
17:	learn: 3930

In [75]:
model_owner = CatBoostRegressor(depth=10, l2_leaf_reg=3, learning_rate=0.03)
model_owner.fit(owner_train_X.values, owner_train_Y.values)

0:	learn: 4057698.2112046	total: 390ms	remaining: 6m 29s
1:	learn: 3974449.2159326	total: 776ms	remaining: 6m 27s
2:	learn: 3891823.9774231	total: 1.14s	remaining: 6m 20s
3:	learn: 3814138.1325939	total: 1.53s	remaining: 6m 21s
4:	learn: 3735887.5370996	total: 1.9s	remaining: 6m 18s
5:	learn: 3660882.3321685	total: 2.27s	remaining: 6m 16s
6:	learn: 3588030.2709793	total: 2.66s	remaining: 6m 17s
7:	learn: 3519301.7076769	total: 3.05s	remaining: 6m 18s
8:	learn: 3453625.0064576	total: 3.43s	remaining: 6m 18s
9:	learn: 3390014.6399083	total: 3.8s	remaining: 6m 16s
10:	learn: 3327141.4862893	total: 4.17s	remaining: 6m 15s
11:	learn: 3262446.1665772	total: 4.54s	remaining: 6m 14s
12:	learn: 3203963.0757288	total: 4.92s	remaining: 6m 13s
13:	learn: 3142912.0281810	total: 5.28s	remaining: 6m 11s
14:	learn: 3084826.0689500	total: 5.41s	remaining: 5m 55s
15:	learn: 3030721.6249537	total: 5.78s	remaining: 5m 55s
16:	learn: 2977925.8574938	total: 6.14s	remaining: 5m 55s
17:	learn: 2926323.7420986

In [76]:
X_test, y_test = df[num_columns].drop('price', axis = 1)[16000: ], df['price'][16000: ]
preds1 = model_invest.predict(X_test)
preds2 = model_owner.predict(X_test)
preds = (preds1 + preds2) / 2

print(f'RMSE: {mean_squared_error(y_test, preds)**0.5}')

RMSE: 2112813.7905244096


Видим, что RMSE еще упал, приятно.

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [134]:
from sklearn.linear_model import HuberRegressor

In [135]:
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [136]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [137]:
invest_train_X = df.loc[np.where(df.product_type == 'Investment')[0], num_columns].drop('price', axis = 1)
invest_train_Y = df.loc[(np.where(df.product_type == 'Investment')[0]), 'price']
imputer.fit(invest_train_X)
invest_train_X = pd.DataFrame(data=imputer.transform(invest_train_X), columns=invest_train_X.columns)

In [138]:
owner_train_X = df.loc[(np.where(df.product_type == 'OwnerOccupier')[0]), num_columns].drop('price', axis = 1)
owner_train_Y = df.loc[(np.where(df.product_type == 'OwnerOccupier')[0]), 'price']
imputer.fit(owner_train_X)
owner_train_X = pd.DataFrame(data=imputer.transform(owner_train_X), columns=owner_train_X.columns)

In [140]:
X_test, y_test = df[num_columns].drop('price', axis = 1)[16000: ], df['price'][16000: ]
imputer.fit(X_test)
X_test = pd.DataFrame(data=imputer.transform(X_test), columns=X_test.columns)

In [141]:
model_1 = CatBoostRegressor(depth=10, l2_leaf_reg=3, learning_rate=0.03)
model_2 = HuberRegressor()

In [142]:
model_1.fit(invest_train_X.values, invest_train_Y.values)
model_2.fit(invest_train_X.values, invest_train_Y.values)

0:	learn: 4826005.2831220	total: 563ms	remaining: 9m 22s
1:	learn: 4756286.7159532	total: 1.18s	remaining: 9m 47s
2:	learn: 4690578.0333280	total: 1.7s	remaining: 9m 26s
3:	learn: 4628834.4684623	total: 2.24s	remaining: 9m 17s
4:	learn: 4568253.7179370	total: 2.77s	remaining: 9m 12s
5:	learn: 4507740.3190539	total: 3.3s	remaining: 9m 7s
6:	learn: 4448369.0037344	total: 3.81s	remaining: 9m
7:	learn: 4392941.9548185	total: 4.35s	remaining: 8m 59s
8:	learn: 4337612.5643719	total: 4.86s	remaining: 8m 55s
9:	learn: 4285620.2167463	total: 5.38s	remaining: 8m 52s
10:	learn: 4240082.1480802	total: 5.88s	remaining: 8m 48s
11:	learn: 4196037.1185498	total: 6.4s	remaining: 8m 46s
12:	learn: 4149006.0034169	total: 6.9s	remaining: 8m 43s
13:	learn: 4106560.6463732	total: 7.41s	remaining: 8m 41s
14:	learn: 4065369.3249315	total: 7.92s	remaining: 8m 39s
15:	learn: 4023185.3801317	total: 8.43s	remaining: 8m 38s
16:	learn: 3980542.3848286	total: 8.96s	remaining: 8m 38s
17:	learn: 3943362.0198396	total:

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


HuberRegressor()

In [143]:
model_owner = CatBoostRegressor(depth=10, l2_leaf_reg=3, learning_rate=0.03)
model_owner.fit(owner_train_X.values, owner_train_Y.values)

0:	learn: 4060708.9573062	total: 352ms	remaining: 5m 52s
1:	learn: 3972976.2519195	total: 738ms	remaining: 6m 8s
2:	learn: 3888283.9119333	total: 1.12s	remaining: 6m 11s
3:	learn: 3809643.6475848	total: 1.49s	remaining: 6m 11s
4:	learn: 3732178.9166374	total: 1.89s	remaining: 6m 16s
5:	learn: 3656639.0431085	total: 2.28s	remaining: 6m 17s
6:	learn: 3587739.9737435	total: 2.67s	remaining: 6m 18s
7:	learn: 3519337.2387332	total: 3.07s	remaining: 6m 20s
8:	learn: 3454351.6431194	total: 3.44s	remaining: 6m 19s
9:	learn: 3387983.2867090	total: 3.87s	remaining: 6m 23s
10:	learn: 3321608.6760196	total: 4.25s	remaining: 6m 22s
11:	learn: 3264133.9096587	total: 4.65s	remaining: 6m 22s
12:	learn: 3204062.7874771	total: 5.05s	remaining: 6m 23s
13:	learn: 3144504.2558022	total: 5.42s	remaining: 6m 21s
14:	learn: 3087652.2837477	total: 5.81s	remaining: 6m 21s
15:	learn: 3033069.2085216	total: 6.17s	remaining: 6m 19s
16:	learn: 2981440.2648533	total: 6.55s	remaining: 6m 18s
17:	learn: 2925969.297082

In [144]:
preds11 = model_1.predict(X_test)
preds12 = model_2.predict(X_test)
preds2 = model_owner.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but HuberRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


В качестве байзлайна использую примитивно коэф-ты 0.5

In [154]:
preds1 = 0.5*preds11 + 0.5*preds12
preds = (preds1 + preds2) / 2

print(f'RMSE: {mean_squared_error(y_test, preds)**0.5}')

RMSE: 2846536.006857003


А теперь напишем небольшой цикл и вычислим RMSE с каждым коэф-том.

In [152]:
d = {}
for x in range(1, 10):
  for y in range(1, 10):
    preds1 = (x/10)*preds11 + (y/10)*preds12
    preds = (preds1 + preds2) / 2
    d[str(x)+str(y)] = mean_squared_error(y_test, preds)**0.5

print(min(d, key=d.get))

91


In [153]:
preds1 = 0.9*preds11 + 0.1*preds12
preds = (preds1 + preds2) / 2

print(f'RMSE: {mean_squared_error(y_test, preds)**0.5}')

RMSE: 2315644.5569649143
